In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow as tf
import pandas as pd
import re


gpt2.download_gpt2(model_name="355M")

gpt2.mount_gdrive()

gpt2.copy_checkpoint_from_gdrive(run_name='run1')

gpt2.copy_file_from_gdrive('gpt2_generate.csv')
df = pd.read_csv('gpt2_generate.csv')


TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 652Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 120Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 355Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:09, 148Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 252Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 84.4Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 107Mit/s]                                                       


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
df.columns = ['tags']
df.tags = df.tags.dropna()

In [3]:
results = []
i = 1200
while i < 1600:
  print(i)
  tf.reset_default_graph()
  with gpt2.start_tf_sess() as sess:
    gpt2.load_gpt2(sess, run_name='run1')
    j = 0
    while j < df.shape[0] and j < 5:
      tag = df.loc[i,'tags']
      value = tag + ' [SEP] '
      try:
        %time text = gpt2.generate(sess, run_name='run1', return_as_list=True, length=200, prefix=value, truncate='<|endoftext|>', nsamples=10, batch_size=10, temperature = 0.7, top_p = 95)
        text_clean = [(tag, re.sub('\n+', '\n',re.sub("&amp;#x200B;", "", t[len(value):])).strip()) for t in text]
        if not text_clean:
          i -= 1
        else:
          results.extend(text_clean)
      except Exception as e:
        print('generation failed! skipping')
        print(e)
      i += 1
      j += 1
fake_out = pd.DataFrame(results, columns = ['tag','synopsis'])
fake_out['real']=0 
fake_out.to_csv(f'bert_gan_fake{i}.csv', index = False)
gpt2.copy_file_to_gdrive(f'bert_gan_fake{i}.csv')

1200
Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
CPU times: user 11.2 s, sys: 307 ms, total: 11.5 s
Wall time: 16.2 s
CPU times: user 11.2 s, sys: 316 ms, total: 11.5 s
Wall time: 14.7 s
CPU times: user 11.6 s, sys: 430 ms, total: 12 s
Wall time: 15.2 s
CPU times: user 11.8 s, sys: 396 ms, total: 12.2 s
Wall time: 15.5 s
CPU times: user 12.2 s, sys: 406 ms, total: 12.6 s
Wall time: 15.8 s
1205
Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000
CPU times: user 11.6 s, sys: 371 ms, total: 12 s
Wall time: 15.2 s
CPU times: user 11.4 s, sys: 388 ms, total: 11.8 s
Wall time: 15.1 s
CPU times: user 11.4 s, sys: 332 ms, total: 11.8 s
Wall time: 15 s
CPU times: user 11.9 s, sys: 366 ms, total: 12.2 s
Wall time: 15.5 s
CPU times: user 12.3 s, sys: 376 ms, total: 12.7 s


In [ ]:
# fake_out = pd.DataFrame(results, columns = ['reply'])
# fake_out['real']=0 
# fake_out.to_csv(f'bert_gan_fake{i}.csv')
# gpt2.copy_file_to_gdrive(f'bert_gan_fake{i}.csv')